<h1 style="text-align:center">Atelier numérique de thermodynamique statistique</h1>
<h2 style="text-align:center">PARTIE 1</h2>

Veuillez indiquer le nom des membres de votre équipe dans la cellule suivante.

# Atelier préparatoire #
### Discussion interdisciplinaire de sujets en probablilité & statistiques ###

**Quelques fondements de mathématiques statistiques:** par exemple bien définir variables aléatoires, échantillon et population totale, estimateurs, fonction de distribution cumulative (répartition), densité de probabilité, moments, etc. - **Programme GPH**

**Les distributions statistiques de particules indiscernables:** en particulier celle de Fermi-Dirac avec les notions de potentiel chimique et d’occupation des états en fonction de la température, en analogie avec le remplissage selon le principe principe d’Aufbau introduit en classe pour les atomes à température nulle. - **Programme PHY**

# Introduction #
Ce travail révise d'abord quelques bases générales de physique statistique classique avec le script `TDSrevision-2Dsimulation.py` qui simule la théorie cinétique des gaz parfaits en 2D. Cette simulation utilise la géométrie de sphères dures, mais dans le cas d'un gaz ici pour introduire des collisions élastiques sur des trajectoires ballistiques de particules. Une sphère est colorée et grossie seulement pour l’effet visuel dans l'animation, remarquez dans le code que la physique de l’algorithme considère bien des particules totalement identiques. Les questions sur cette simulation, à répondre directement dans les cellules du _Notebook_ ici-même, explorent quelques paramètres de la thermodynamique statistique et introduisent de nouveaux termes utiles à l'étude de la dynamique des électrons dans la matière.

_N.B._ 
- _Pour montrer les animations à l'écran, le script `TDSrevision-2Dsimulation.py` importe la librairie `VPython` qu'il faut donc installer. Des liens vers sa documentation et de l'information complémentaire sont donnés dans la médiagraphie à la fin._
- _Le code dans ce script est abusivement commenté dans notre contexte pédagogique, mais il serait bien sûr préférable de s’en tenir aux recommandations du <a href="https://www.python.org/dev/peps/pep-0008"> PEP 8 — Style Guide for Python Code</a>._
- _Notez finalement que la boucle principale à la fin du script laisse l'utilisateur voir l'animation aussi longtemps que souhaité, assurez-vous donc de savoir comment l'interrompre correctement avant de lancer la simulation ou de la remplacer par une boucle `for`._

# Théorie cinétique des gaz parfaits #

### Simulation 2D ###

<i>Note 1: Puisque la commande %un TDSrevion-2Dsimulation.py ne fonctionnait pas, on a recopié le fichier dans la première cellule.<i>

In [2]:
from vpython import *
import numpy as np
import math
import matplotlib.pyplot as plt

# win = 500 # peut aider à définir la taille d'un autre objet visuel comme un histogramme proportionnellement à la taille du canevas.

# Déclaration de variables influençant le temps d'exécution de la simulation
Natoms = 200  # change this to have more or fewer atoms
dt = 1E-5  # pas d'incrémentation temporel

# Déclaration de variables physiques "Typical values"
mass = 4E-3/6E23 # helium mass
Ratom = 0.01 # wildly exaggerated size of an atom
k = 1.4E-23 # Boltzmann constant # TODO: changer pour une constante de Boltzmann en eV/K :)
T = 300 # around room temperature

#### CANEVAS DE FOND ####
L = 1 # container is a cube L on a side
gray = color.gray(0.7) # color of edges of container and spheres below
animation = canvas( width=750, height=500) # , align='left')
animation.range = L
# animation.title = 'Théorie cinétique des gaz parfaits'
# s = """  Simulation de particules modélisées en sphères dures pour représenter leur trajectoire ballistique avec collisions. Une sphère est colorée et grossie seulement pour l’effet visuel permettant de suivre sa trajectoire plus facilement dans l'animation, sa cinétique est identique à toutes les autres particules.

# """
# animation.caption = s

#### ARÊTES DE BOÎTE 2D ####
d = L/2+Ratom
r = 0.005
cadre = curve(color=gray, radius=r)
cadre.append([vector(-d,-d,0), vector(d,-d,0), vector(d,d,0), vector(-d,d,0), vector(-d,-d,0)])

#### POSITION ET QUANTITÉ DE MOUVEMENT INITIALE DES SPHÈRES ####
Atoms = [] # Objet qui contiendra les sphères pour l'animation
p = [] # quantité de mouvement des sphères
apos = [] # position des sphères
pavg = sqrt(2*mass*1.5*k*T) # average kinetic energy p**2/(2mass) = (3/2)kT : Principe de l'équipartition de l'énergie en thermodynamique statistique classique # TODO: Changer pour quantités de mouvement initiales aléatoires sur une plage raisonnable cohérente avec température pièce

for i in range(Natoms):
    x = L*random()-L/2 # position aléatoire qui tient compte que l'origine est au centre de la boîte
    y = L*random()-L/2
    z = 0
    if i == 0:  # garde une sphère plus grosse et colorée parmis toutes les grises
        Atoms.append(simple_sphere(pos=vector(x,y,z), radius=0.03, color=color.magenta)) #, make_trail=True, retain=100, trail_radius=0.3*Ratom))
    else: Atoms.append(simple_sphere(pos=vector(x,y,z), radius=Ratom, color=gray))
    apos.append(vec(x,y,z)) # liste de la position initiale de toutes les sphères
#    theta = pi*random() # direction de coordonnées sphériques, superflue en 2D
    phi = 2*pi*random() # direction aléatoire pour la quantité de mouvement
    px = pavg*cos(phi)  # qte de mvt initiale selon l'équipartition
    py = pavg*sin(phi)
    pz = 0
    p.append(vector(px,py,pz)) # liste de la quantité de mouvement initiale de toutes les sphères

#### FONCTION POUR IDENTIFIER LES COLLISIONS, I.E. LORSQUE LA DISTANCE ENTRE LES CENTRES DE 2 SPHÈRES EST À LA LIMITE DE S'INTERPÉNÉTRER ####
def checkCollisions():
    hitlist = []   # initialisation
    r2 = 2*Ratom   # distance critique où les 2 sphères entre en contact à la limite de leur rayon
    r2 *= r2   # produit scalaire pour éviter une comparaison vectorielle ci-dessous
    for i in range(Natoms):
        ai = apos[i]
        for j in range(i) :
            aj = apos[j]
            dr = ai - aj   # la boucle dans une boucle itère pour calculer cette distance vectorielle dr entre chaque paire de sphère
            if mag2(dr) < r2:   # test de collision où mag2(dr) qui retourne la norme élevée au carré de la distance intersphère dr
                hitlist.append([i,j]) # liste numérotant toutes les paires de sphères en collision
    return hitlist

#### BOUCLE PRINCIPALE POUR L'ÉVOLUTION TEMPORELLE DE PAS dt ####
## ATTENTION : la boucle laisse aller l'animation aussi longtemps que souhaité, assurez-vous de savoir comment interrompre vous-même correctement (souvent `ctrl+c`, mais peut varier)
## ALTERNATIVE : vous pouvez bien sûr remplacer la boucle "while" par une boucle "for" avec un nombre d'itérations suffisant pour obtenir une bonne distribution statistique à l'équilibre

for r in range(0,100):
    rate(50)  # limite la vitesse de calcul de la simulation pour que l'animation soit visible à l'oeil humain!

    #### DÉPLACE TOUTES LES SPHÈRES D'UN PAS SPATIAL deltax
    vitesse = []   # vitesse instantanée de chaque sphère
    deltax = []  # pas de position de chaque sphère correspondant à l'incrément de temps dt
    for i in range(Natoms):
        vitesse.append(p[i]/mass)   # par définition de la quantité de nouvement pour chaque sphère
        deltax.append(vitesse[i] * dt)   # différence avant pour calculer l'incrément de position
        Atoms[i].pos = apos[i] = apos[i] + deltax[i]  # nouvelle position de l'atome après l'incrément de temps dt

    #### CONSERVE LA QUANTITÉ DE MOUVEMENT AUX COLLISIONS AVEC LES MURS DE LA BOÎTE ####
    for i in range(Natoms):
        loc = apos[i]
        if abs(loc.x) > L/2:
            if loc.x < 0: p[i].x =  abs(p[i].x)  # renverse composante x au mur de gauche
            else: p[i].x =  -abs(p[i].x)   # renverse composante x au mur de droite
        if abs(loc.y) > L/2:
            if loc.y < 0: p[i].y = abs(p[i].y)  # renverse composante y au mur du bas
            else: p[i].y =  -abs(p[i].y)  # renverse composante y au mur du haut

    #### LET'S FIND THESE COLLISIONS!!! ####
    hitlist = checkCollisions()

    #### CONSERVE LA QUANTITÉ DE MOUVEMENT AUX COLLISIONS ENTRE SPHÈRES ####
    for ij in hitlist:

        # définition de nouvelles variables pour chaque paire de sphères en collision
        i = ij[0]  # extraction du numéro des 2 sphères impliquées à cette itération
        j = ij[1]
        ptot = p[i]+p[j]   # quantité de mouvement totale des 2 sphères
        mtot = 2*mass    # masse totale des 2 sphères
        Vcom = ptot/mtot   # vitesse du référentiel barycentrique/center-of-momentum (com) frame
        posi = apos[i]   # position de chacune des 2 sphères
        posj = apos[j]
        vi = p[i]/mass   # vitesse de chacune des 2 sphères
        vj = p[j]/mass
        rrel = posi-posj  # vecteur pour la distance entre les centres des 2 sphères
        vrel = vj-vi   # vecteur pour la différence de vitesse entre les 2 sphères

        # exclusion de cas où il n'y a pas de changements à faire
        if vrel.mag2 == 0: continue  # exactly same velocities si et seulement si le vecteur vrel devient nul, la trajectoire des 2 sphères continue alors côte à côte
        if rrel.mag > Ratom: continue  # one atom went all the way through another, la collision a été "manquée" à l'intérieur du pas deltax # TODO: ajouter une boucle fractionnant dt à une limite raisonnable pour aller chercher la collision manquée

        # calcule la distance et temps d'interpénétration des sphères dures qui ne doit pas se produire dans ce modèle # TODO: donner géométrie en devoir en illustrant avec Geogebra -> Tikz et/ou animation wiki https://en.wikipedia.org/wiki/Elastic_collision#Two-dimensional_collision_with_two_moving_objects
        dx = dot(rrel, vrel.hat)       # rrel.mag*cos(theta) où theta is the angle between vrel and rrel:
        dy = cross(rrel, vrel.hat).mag # rrel.mag*sin(theta)
        alpha = asin(dy/(2*Ratom))  # alpha is the angle of the triangle composed of rrel, path of atom j, and a line from the center of atom i to the center of atom j where atome j hits atom i
        d = (2*Ratom)*cos(alpha)-dx # distance traveled into the atom from first contact
        deltat = d/vrel.mag         # time spent moving from first contact to position inside atom

        #### CHANGE L'INTERPÉNÉTRATION DES SPHÈRES PAR LA CINÉTIQUE DE COLLISION ####
        posi = posi-vi*deltat   # back up to contact configuration
        posj = posj-vj*deltat
        pcomi = p[i]-mass*Vcom  # transform momenta to center-of-momentum (com) frame
        pcomj = p[j]-mass*Vcom
        rrel = hat(rrel)    # vecteur unitaire aligné avec rrel
        pcomi = pcomi-2*dot(pcomi,rrel)*rrel # bounce in center-of-momentum (com) frame
        pcomj = pcomj-2*dot(pcomj,rrel)*rrel  # TODO: convertir avec masse réduite et vitesse du centre de masse en corrigeant les unités
        p[i] = pcomi+mass*Vcom # transform momenta back to lab frame
        p[j] = pcomj+mass*Vcom
        apos[i] = posi+(p[i]/mass)*deltat # move forward deltat in time, ramenant au même temps où sont rendues les autres sphères dans l'itération
        apos[j] = posj+(p[j]/mass)*deltat

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Questions statistiques ###

I. _(3 points)_  &mdash; Utilisez la liste finale des vecteurs de quantité de mouvement $\vec{p}$ de toutes les sphères pour trouver la moyenne de son carré $\langle p^2\rangle=\langle\vec{p}\cdot\vec{p}\rangle$ et l'imprimer avec la fonction `print()` dans la cellule qui suit. 

In [ ]:

dotp=[] # liste vide qui va contenir le produit scalaire de tous les vecteurs de la liste entre eux-mêmes
for i in range(len(p)):
    dotp.append(dot(p[i],p[i]))

dotp_moy=sum(dotp)/len(dotp)

print('Moyenne du carré de la quantité de mouvement: '+str(dotp_moy)+' kg*m/s')

NameError: name 'p' is not defined


II. _(2 points)_  &mdash; La température $T$ (macroscopique) est proportionnelle à l'énergie cinétique moyenne de l'ensemble des particules et peut se calculer classiquement selon le <a href="https://fr.wikipedia.org/wiki/%C3%89quipartition_de_l%27%C3%A9nergie">principe d'équipartition</a> qui s'écrit ainsi en tenant compte des 3 degrés de liberté en translation de chaque particule de masse $m$:
\begin{equation}
\frac{\langle\vec{p}^2\rangle}{2m}=\frac{3}{2}k_BT,
\end{equation}
où $k_B$ est la constante de Boltzmann. Quelle est la température du gaz de sphères dures à la fin de la simulation? Est-ce qu'elle a changé significativement par rapport à sa valeur initiale?

In [ ]:
print('Température retrouvée: '+str(round((2*dotp_moy)/(3*2*mass*k),5))+' K')

III. _(10 points)_ &mdash; Modifiez le code de la simulation pour ajouter une fonction qui suit la trajectoire d'UNE SEULE particule, c'est-à-dire qu'elle doit enregistrer dans une liste des valeurs de variables pour cette particule à chacune de ses collisions. Les deux variables scalaires à lister sont:
- la distance que la particule a parcouru entre chaque collision,
- le temps écoulé entre ces collisions.

Copiez le code de votre fonction dans la cellule qui suit en y commentant clairement les variables pour ces listes qui devront persister après avoir interrompu l'exécution de la simulation. N'oubliez pas d'inclure votre fichier Python (`.py`) modifié avec la simulation complète lors de la remise.

<i>Note 2: La fonction lance une deuxième simulation, mais des modifications ont été apportées au code afin de récolter les informations d'une seule particule (la rose) tout au long de l'expérience. Après discussion avec Claudine, comme il ne s'agit pas d'un cours de physique numérique, elle n'y voit pas de problème. Les commentaires débutant par "AJOUT:" sont relatifs aux modifications apportées.<i>

In [ ]:

def particule_seule():
    #### CANEVAS DE FOND ####
    L = 1 # container is a cube L on a side
    gray = color.gray(0.7) # color of edges of container and spheres below
    animation = canvas( width=750, height=500) # , align='left')
    animation.range = L
    # animation.title = 'Théorie cinétique des gaz parfaits'
    # s = """  Simulation de particules modélisées en sphères dures pour représenter leur trajectoire ballistique avec collisions. Une sphère est colorée et grossie seulement pour l’effet visuel permettant de suivre sa trajectoire plus facilement dans l'animation, sa cinétique est identique à toutes les autres particules.

    # """
    # animation.caption = s

    #### ARÊTES DE BOÎTE 2D ####
    d = L/2+Ratom
    r = 0.005
    cadre = curve(color=gray, radius=r)
    cadre.append([vector(-d,-d,0), vector(d,-d,0), vector(d,d,0), vector(-d,d,0), vector(-d,-d,0)])

    #### POSITION ET QUANTITÉ DE MOUVEMENT INITIALE DES SPHÈRES ####
    Atoms = [] # Objet qui contiendra les sphères pour l'animation
    p = [] # quantité de mouvement des sphères
    apos = [] # position des sphères
    pavg = sqrt(2*mass*1.5*k*T) # average kinetic energy p**2/(2mass) = (3/2)kT : Principe de l'équipartition de l'énergie en thermodynamique statistique classique # TODO: Changer pour quantités de mouvement initiales aléatoires sur une plage raisonnable cohérente avec température pièce

    for i in range(Natoms):
        x = L*random()-L/2 # position aléatoire qui tient compte que l'origine est au centre de la boîte
        y = L*random()-L/2
        z = 0
        if i == 0:  # garde une sphère plus grosse et colorée parmis toutes les grises
            Atoms.append(simple_sphere(pos=vector(x,y,z), radius=0.03, color=color.magenta)) #, make_trail=True, retain=100, trail_radius=0.3*Ratom))
        else: Atoms.append(simple_sphere(pos=vector(x,y,z), radius=Ratom, color=gray))
        apos.append(vec(x,y,z)) # liste de la position initiale de toutes les sphères
    #    theta = pi*random() # direction de coordonnées sphériques, superflue en 2D
        phi = 2*pi*random() # direction aléatoire pour la quantité de mouvement
        px = pavg*cos(phi)  # qte de mvt initiale selon l'équipartition
        py = pavg*sin(phi)
        pz = 0
        p.append(vector(px,py,pz)) # liste de la quantité de mouvement initiale de toutes les sphères


        #### FONCTION POUR IDENTIFIER LES COLLISIONS, I.E. LORSQUE LA DISTANCE ENTRE LES CENTRES DE 2 SPHÈRES EST À LA LIMITE DE S'INTERPÉNÉTRER ####
    def checkCollisions_seule(r,iteration_coll):
        hitlist = []   # initialisation
        r2 = 2*Ratom   # distance critique où les 2 sphères entre en contact à la limite de leur rayon
        r2 *= r2   # produit scalaire pour éviter une comparaison vectorielle ci-dessous
        for i in range(Natoms):
            ai = apos[i]
            for j in range(i) :
                aj = apos[j]
                dr = ai - aj   # la boucle dans une boucle itère pour calculer cette distance vectorielle dr entre chaque paire de sphère
                if mag2(dr) < r2:   # test de collision où mag2(dr) qui retourne la norme élevée au carré de la distance intersphère dr
                    hitlist.append([i,j]) # liste numérotant toutes les paires de sphères en collision
        if (", 0]" in ''.join(str(i) for i in hitlist))==True: # AJOUT: une fois toutes les collisions répertoriées, vérifie si l'atome 0 a subit au moins une collision dans l'itération r. Si oui, on l'ajoute à la liste des itérations collisionnelles.
            iteration_coll.append(r) # AJOUT
        return hitlist,iteration_coll # AJOUT: retourne un tuple comprenant la liste des collisions totale et la liste des itérations pour lesquels au moins une collision de l'atome 0 est enregistrée.

    #### BOUCLE PRINCIPALE POUR L'ÉVOLUTION TEMPORELLE DE PAS dt ####
    ## ATTENTION : la boucle laisse aller l'animation aussi longtemps que souhaité, assurez-vous de savoir comment interrompre vous-même correctement (souvent `ctrl+c`, mais peut varier)
    ## ALTERNATIVE : vous pouvez bien sûr remplacer la boucle "while" par une boucle "for" avec un nombre d'itérations suffisant pour obtenir une bonne distribution statistique à l'équilibre

    #Objectif: suivre l'atome 0
    iteration_coll=[] # AJOUT: iterations à lesquelles l'atome 0 subit une collision
    iteration_vitesse=[] # AJOUT: vitesse de la particule 0 à chaque itération  

    for r in range(400): #nombre d'itération de pas dt
        rate(25)  # limite la vitesse de calcul de la simulation pour que l'animation soit visible à l'oeil humain!

        #### DÉPLACE TOUTES LES SPHÈRES D'UN PAS SPATIAL deltax
        vitesse = []   # vitesse instantanée de chaque sphère
        deltax = []  # pas de position de chaque sphère correspondant à l'incrément de temps dt
        for i in range(Natoms):
            vitesse.append(p[i]/mass)   # par définition de la quantité de nouvement pour chaque sphère
            deltax.append(vitesse[i] * dt)   # différence avant pour calculer l'incrément de position
            Atoms[i].pos = apos[i] = apos[i] + deltax[i]  # nouvelle position de l'atome après l'incrément de temps dt
            if i==0:
                iteration_vitesse.append(vitesse[i]) # AJOUT: sauvegarde la vitesse de la particule 0 à chaque itération

        #### CONSERVE LA QUANTITÉ DE MOUVEMENT AUX COLLISIONS AVEC LES MURS DE LA BOÎTE ####
        for i in range(Natoms):
            loc = apos[i]
            if abs(loc.x) > L/2:
                if loc.x < 0: p[i].x =  abs(p[i].x)  # renverse composante x au mur de gauche
                else: p[i].x =  -abs(p[i].x)   # renverse composante x au mur de droite
                if i==0:
                    iteration_coll.append(r) # AJOUT: stockage de l'itération à laquelle la sphère rose frappe un mur (gauche ou droite)
            if abs(loc.y) > L/2:
                if loc.y < 0: p[i].y = abs(p[i].y)  # renverse composante y au mur du bas
                else: p[i].y =  -abs(p[i].y)  # renverse composante y au mur du haut
                if i==0:
                    iteration_coll.append(r) # AJOUT: stockage de l'itération à laquelle la sphère rose frappe un mur (haut ou bas)
                    
        #### LET'S FIND THESE COLLISIONS!!! ####
        checkcollision_tuple = checkCollisions_seule(r,iteration_coll)
        hitlist = checkcollision_tuple[0]
        iteration_coll=checkcollision_tuple[1]

        #### CONSERVE LA QUANTITÉ DE MOUVEMENT AUX COLLISIONS ENTRE SPHÈRES ####
        for ij in hitlist:

            # définition de nouvelles variables pour chaque paire de sphères en collision
            i = ij[0]  # extraction du numéro des 2 sphères impliquées à cette itération
            j = ij[1]
            ptot = p[i]+p[j]   # quantité de mouvement totale des 2 sphères
            mtot = 2*mass    # masse totale des 2 sphères
            Vcom = ptot/mtot   # vitesse du référentiel barycentrique/center-of-momentum (com) frame
            posi = apos[i]   # position de chacune des 2 sphères
            posj = apos[j]
            vi = p[i]/mass   # vitesse de chacune des 2 sphères
            vj = p[j]/mass
            rrel = posi-posj  # vecteur pour la distance entre les centres des 2 sphères
            vrel = vj-vi   # vecteur pour la différence de vitesse entre les 2 sphères

            # exclusion de cas où il n'y a pas de changements à faire
            if vrel.mag2 == 0: continue  # exactly same velocities si et seulement si le vecteur vrel devient nul, la trajectoire des 2 sphères continue alors côte à côte
            if rrel.mag > Ratom: continue  # one atom went all the way through another, la collision a été "manquée" à l'intérieur du pas deltax # TODO: ajouter une boucle fractionnant dt à une limite raisonnable pour aller chercher la collision manquée

            # calcule la distance et temps d'interpénétration des sphères dures qui ne doit pas se produire dans ce modèle # TODO: donner géométrie en devoir en illustrant avec Geogebra -> Tikz et/ou animation wiki https://en.wikipedia.org/wiki/Elastic_collision#Two-dimensional_collision_with_two_moving_objects
            dx = dot(rrel, vrel.hat)       # rrel.mag*cos(theta) où theta is the angle between vrel and rrel:
            dy = cross(rrel, vrel.hat).mag # rrel.mag*sin(theta)
            alpha = asin(dy/(2*Ratom))  # alpha is the angle of the triangle composed of rrel, path of atom j, and a line from the center of atom i to the center of atom j where atome j hits atom i
            d = (2*Ratom)*cos(alpha)-dx # distance traveled into the atom from first contact
            deltat = d/vrel.mag         # time spent moving from first contact to position inside atom

            #### CHANGE L'INTERPÉNÉTRATION DES SPHÈRES PAR LA CINÉTIQUE DE COLLISION ####
            posi = posi-vi*deltat   # back up to contact configuration
            posj = posj-vj*deltat
            pcomi = p[i]-mass*Vcom  # transform momenta to center-of-momentum (com) frame
            pcomj = p[j]-mass*Vcom
            rrel = hat(rrel)    # vecteur unitaire aligné avec rrel
            pcomi = pcomi-2*dot(pcomi,rrel)*rrel # bounce in center-of-momentum (com) frame
            pcomj = pcomj-2*dot(pcomj,rrel)*rrel  # TODO: convertir avec masse réduite et vitesse du centre de masse en corrigeant les unités
            p[i] = pcomi+mass*Vcom # transform momenta back to lab frame
            p[j] = pcomj+mass*Vcom
            apos[i] = posi+(p[i]/mass)*deltat # move forward deltat in time, ramenant au même temps où sont rendues les autres sphères dans l'itération
            apos[j] = posj+(p[j]/mass)*deltat


    nb_iteration_entre_coll = [j-i for i, j in zip(iteration_coll[:-1], iteration_coll[1:])] # AJOUT: nombre d'itérations entre deux itérations correspondantes à une collision de l'atome 0
    temps_entre_coll = [i * dt for i in nb_iteration_entre_coll] # AJOUT: le temps entre deux collisions est obtenu en multipliant le nombre d'itérations entre deux collisions par le temps dt écoulé entre deux itérations

    vitesse_coll_mag = [mag(iteration_vitesse[i]) for i in iteration_coll] # AJOUT: extrait la magnitude des vecteurs vitesses de la particule 0 à chaque collision
    vitesse_coll_vecteur = [iteration_vitesse[i] for i in iteration_coll] # AJOUT: extrait les vecteurs vitesses de la particule 0 chaque collision
    deplacement_entre_coll = [i * j for i,j in zip(vitesse_coll_mag, temps_entre_coll)] # AJOUT: calcul le déplacement entre les collisions en multipliant le temps écoulé par la vitesse x=vt

    temps_moy=sum(temps_entre_coll)/len(temps_entre_coll) # AJOUT: effectue la moyenne des durées entre les collisions
    deplacement_moy=sum(deplacement_entre_coll)/len(deplacement_entre_coll) # AJOUT: calcul la moyenne des déplacements inter collisionnels

    return deplacement_entre_coll,temps_entre_coll, deplacement_moy, temps_moy, vitesse_coll_mag, vitesse_coll_vecteur # AJOUT: retourne des listes utiles pour la suite


IV. _(2 points)_ &mdash; Calculez le **libre parcours moyen** $l_{moy}$ et le **temps de collision** $\tau$ qui sont les valeurs moyennes des deux listes compilées à l’étape précédente.

_Pour votre information, le libre parcours moyen est de l’ordre de 100 nm dans l'air à température et pression ambiantes, mais_ $l_{moy}$ _peut dépasser 100 000 km dans une enceinte sous vide avec les technologies de pompes modernes!_



In [ ]:

resultats_particule_seule = particule_seule()

print('Libre parcours moyen de la particule seule: '+str(round(resultats_particule_seule[2],5))+' m')
print('Temps moyen entre les collisions de la particule seule: '+str(round(resultats_particule_seule[3],5))+' s')

V. _(2 points)_ Calculez la vitesse $\vec{v}$ de la particule entre chaque paire de collisions. Quelle est la vitesse moyenne $\langle\vec{v}\rangle$ de la particule?

In [ ]:

vitesse_entre_coll_seule = [i/j for i, j in zip(resultats_particule_seule[0], resultats_particule_seule[1])] # v=x/t pour chaque "entre collisions"
vitesse_moyenne_particule_seule = sum(vitesse_entre_coll_seule)/len(vitesse_entre_coll_seule)

print('Vitesse moyenne de la particule seule: '+str(round(vitesse_moyenne_particule_seule,5))+' m/s')

VI. _(5 points)_ &mdash; Pour cette même liste de vitesses, comparez les distributions de la norme $||\vec{v}||$, du carré $v^2$ et d’une de ses composantes $v_x^2$ en étalonnant l’abscisse pour contraster les histogrammes avec une échelle appropriée. Indiquez sur ce graphique la moyenne, le mode, la médiane et la moyenne quadratique des distributions.

In [ ]:

vitesse_entre_coll_mag_seule=resultats_particule_seule[4]
vitesse_entre_coll_vecteur_seule=resultats_particule_seule[5]
vitesse_entre_coll_carre_seule=[dot(vitesse_entre_coll_vecteur_seule[i],vitesse_entre_coll_vecteur_seule[i]) for i in range(0,len(vitesse_entre_coll_vecteur_seule))]

vitesse_entre_coll_x_seule = [vitesse_entre_coll_vecteur_seule[i].x for i in range(len(vitesse_entre_coll_vecteur_seule))]

#print(vitesse_entre_coll_mag_seule)
#print(vitesse_entre_coll_carre_seule)
#print(vitesse_entre_coll_x_seule)

fig1=plt.figure()

fig1, axs = plt.subplots(3, 1)

axs[0].hist(vitesse_entre_coll_mag_seule, bins=20, edgecolor='black', color='red')
axs[0].title.set_text('Module de la vitesse ||v||')

axs[1].hist(vitesse_entre_coll_carre_seule, bins=20, edgecolor='black', color='green')
axs[1].title.set_text('Module de la vitesse au carré ||v^2||')
axs[1].set_ylabel("Quantité d'inter collisions")

axs[2].hist(vitesse_entre_coll_x_seule, bins=20, edgecolor='black', color='blue')
axs[2].title.set_text('Module de la vitesse en x ||v_x||')
axs[2].set_xlabel("Vitesse [m/s]")

plt.tight_layout()

#fig1.savefig("/Users/david.tremblay7/Desktop/histo_vitesses.pdf", bbox_inches='tight')

plt.show()

# Médiagraphie #
 - La simulation utilise la librairie <a href="https://vpython.org">VPython</a> conçue pour faciliter la visualisation de physique en 3D, avec les instructions d’installation <a href="https://vpython.org/presentation2018/install.html">ici</a> et la documentation <a href="https://www.glowscript.org/docs/VPythonDocs/index.html">ici</a>. Le code adapte en 2D et commente en détail l’exemple <a href="https://www.glowscript.org/#/user/GlowScriptDemos/folder/Examples/program/HardSphereGas-VPython">HardSphereGas-VPython</a> du site interactif <a href="https://www.glowscript.org">GlowScript</a> pour programmer des animations avec VPython directement en ligne.